# Recipe Nutrition Calculator & Modifier

An AI-powered tool that:
1. Scrapes recipes from popular recipe websites (AllRecipes, Food Network, BBC Good Food, etc.)
2. Calculates approximate nutrition information using LLM
3. Suggests healthy ingredient substitutions
4. Modifies recipes for dietary restrictions (vegan, keto, gluten-free, etc.)

## Features
- 🌐 Web recipe scraping from multiple sources
- 📊 Nutrition estimation (calories, macros, vitamins, minerals)
- 🔄 Dietary modifications (vegan, keto, gluten-free, low-sodium, etc.)
- 💡 Healthy substitution suggestions
- 📏 Serving size adjustments


In [ ]:
# Import required libraries
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

# Import our custom modules
from recipe_scraper import RecipeScraper
from recipe_formatter import format_ingredients_for_prompt, format_ingredients_simple, format_instructions_for_prompt, format_nutrition_for_prompt
from display_utils import display_nutrition, display_substitutions, display_modified_recipe
from config import MODEL


## Step 1: Environment Setup & API Configuration

In [ ]:
# Load environment variables from .env file
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Validate the OpenAI API key
if not api_key:
    print("⚠️ No API key found! Please create a .env file with: OPENAI_API_KEY=your_key_here")
elif not api_key.startswith("sk-proj-") and not api_key.startswith("sk-"):
    print("⚠️ API key format looks incorrect")
else:
    print("✅ API key found and looks good!")

# Initialize OpenAI client
openai = OpenAI()

# Model configuration
MODEL = "gpt-4o-mini"


## Step 2: LLM Functions - All API Calls

All OpenAI API calls are centralized here. Utility functions are imported from separate modules.


### LLM Function 1: Parse Recipe Structure


In [ ]:
def parse_recipe_with_llm(recipe_text_or_url):
    """Use LLM to extract structured recipe data from text or URL."""
    
    system_prompt = """You are a recipe parser expert. Extract structured recipe information from text.
Return a JSON object with this exact structure:
{
    "title": "Recipe name",
    "servings": number of servings (integer, or null if not found),
    "ingredients": [
        {"ingredient": "ingredient name", "quantity": "amount with unit", "notes": "any preparation notes"}
    ],
    "instructions": ["step 1", "step 2", ...]
}

Be precise with ingredient quantities. Include units (cups, tbsp, grams, etc.).
If recipe is from a URL, extract the actual recipe content, ignoring navigation and ads."""

    user_prompt = f"""Extract the recipe information from this content:

{recipe_text_or_url}

Return ONLY valid JSON, no markdown formatting or additional text."""
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )
    
    return json.loads(response.choices[0].message.content)


### LLM Function 2: Calculate Nutrition

In [ ]:
def calculate_nutrition(recipe_data):
    """Calculate nutrition information for a recipe using LLM."""
    
    ingredients_text = format_ingredients_for_prompt(recipe_data)
    servings = recipe_data.get('servings', 1)
    
    system_prompt = """You are a nutrition expert. Calculate approximate nutrition information for recipes.
Provide estimates based on your knowledge of food nutrition values.

Return a JSON object with this structure:
{
    "total_recipe": {
        "calories": number,
        "protein_g": number,
        "carbohydrates_g": number,
        "fat_g": number,
        "fiber_g": number,
        "sugar_g": number,
        "sodium_mg": number,
        "saturated_fat_g": number
    },
    "per_serving": {
        "calories": number,
        "protein_g": number,
        "carbohydrates_g": number,
        "fat_g": number,
        "fiber_g": number,
        "sugar_g": number,
        "sodium_mg": number,
        "saturated_fat_g": number
    },
    "key_findings": ["finding 1", "finding 2", ...],
    "ingredient_breakdown": [
        {"ingredient": "name", "calories": number, "protein_g": number, "carbs_g": number, "fat_g": number}
    ]
}

Provide realistic estimates. If unsure about specific quantities, make reasonable approximations.
Include 2-4 key findings about the recipe's nutritional profile."""
    
    user_prompt = f"""Calculate nutrition for this recipe:

Recipe: {recipe_data.get('title', 'Unknown')}
Servings: {servings}

Ingredients:
{ingredients_text}

Calculate total nutrition for the entire recipe, then divide by {servings} for per-serving values.
Provide ingredient-level breakdown for top 5-8 major ingredients."""
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )
    
    nutrition_data = json.loads(response.choices[0].message.content)
    nutrition_data['servings'] = servings
    return nutrition_data


### LLM Function 3: Suggest Healthy Substitutions


In [ ]:
def suggest_substitutions(recipe_data, nutrition_data):
    """Suggest healthy ingredient substitutions to improve nutrition profile."""
    
    ingredients_text = format_ingredients_simple(recipe_data)
    nutrition_text = format_nutrition_for_prompt(nutrition_data)
    
    system_prompt = """You are a nutritionist and cooking expert. Suggest healthy ingredient substitutions.
For each substitution, explain why it's healthier and how it affects taste/texture.

Return JSON with this structure:
{
    "substitutions": [
        {
            "original": "ingredient with quantity",
            "suggestion": "replacement ingredient with quantity",
            "health_benefit": "why this is healthier",
            "impact": "how it affects taste/texture (brief)",
            "nutrition_improvement": "expected nutrition change"
        }
    ],
    "priority": ["most impactful substitution first", ...]
}

Focus on substitutions that significantly improve health (reduce saturated fat, sugar, sodium, increase fiber, etc.).
Limit to 5-7 best substitutions."""
    
    user_prompt = f"""Suggest healthy substitutions for this recipe:

{recipe_data.get('title', 'Recipe')}

Ingredients:
{ingredients_text}

Current Nutrition (per serving):
{nutrition_text}

Suggest substitutions that improve the nutrition profile."""
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )
    
    return json.loads(response.choices[0].message.content)


### LLM Function 4: Modify Recipe for Dietary Restrictions


In [ ]:
def modify_recipe_for_diet(recipe_data, dietary_restrictions):
    """Modify recipe to comply with dietary restrictions."""
    
    restrictions_text = ", ".join(dietary_restrictions)
    ingredients_text = format_ingredients_for_prompt(recipe_data)
    instructions_text = format_instructions_for_prompt(recipe_data)
    
    system_prompt = f"""You are a culinary expert specializing in dietary modifications.
Modify recipes to comply with these restrictions: {restrictions_text}

Return JSON with this structure:
{{
    "modified_title": "recipe name with dietary note",
    "modifications_summary": "brief summary of changes made",
    "modified_ingredients": [
        {{"ingredient": "name", "quantity": "amount", "notes": "notes", "substitution": "what was changed from"}}
    ],
    "modified_instructions": ["step 1", "step 2", ...],
    "key_changes": ["change 1", "change 2", ...],
    "notes": "any important cooking notes for the modified recipe"
}}

Maintain recipe integrity and flavor profile as much as possible.
If an ingredient cannot be substituted, note it in the summary.
Provide specific substitute ingredients with quantities."""
    
    user_prompt = f"""Modify this recipe for: {restrictions_text}

Recipe: {recipe_data.get('title', '')}
Servings: {recipe_data.get('servings', 1)}

Original Ingredients:
{ingredients_text}

Original Instructions:
{instructions_text}

Create a modified version that complies with all specified dietary restrictions."""
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )
    
    modified_recipe = json.loads(response.choices[0].message.content)
    modified_recipe['original_servings'] = recipe_data.get('servings', 1)
    return modified_recipe


## Step 3: Orchestration Functions

These functions combine LLM calls and utility functions to provide complete workflows.


### Main Analysis Functions


In [ ]:
def analyze_recipe_from_url(url, include_substitutions=True):
    """Complete analysis pipeline: scrape → parse → calculate nutrition → suggest substitutions."""
    print(f"🌐 Scraping recipe from: {url}")
    scraper = RecipeScraper(url)
    
    print("📝 Parsing recipe structure...")
    recipe_data = parse_recipe_with_llm(scraper.get_recipe_text())
    
    print("🧮 Calculating nutrition...")
    nutrition_data = calculate_nutrition(recipe_data)
    
    substitutions_data = None
    if include_substitutions:
        print("💡 Generating substitution suggestions...")
        substitutions_data = suggest_substitutions(recipe_data, nutrition_data)
    
    print("✅ Analysis complete!")
    return {
        'recipe': recipe_data,
        'nutrition': nutrition_data,
        'substitutions': substitutions_data,
        'url': url
    }

def analyze_recipe_from_text(recipe_text, include_substitutions=True):
    """Analyze recipe from plain text input."""
    print("📝 Parsing recipe from text...")
    recipe_data = parse_recipe_with_llm(recipe_text)
    
    print("🧮 Calculating nutrition...")
    nutrition_data = calculate_nutrition(recipe_data)
    
    substitutions_data = None
    if include_substitutions:
        print("💡 Generating substitution suggestions...")
        substitutions_data = suggest_substitutions(recipe_data, nutrition_data)
    
    print("✅ Analysis complete!")
    return {
        'recipe': recipe_data,
        'nutrition': nutrition_data,
        'substitutions': substitutions_data
    }


## Usage Examples

### Example 1: Analyze Recipe from URL


In [ ]:
# Example: Analyze a recipe from AllRecipes or any recipe website
# Add your recipe URL:

recipe_url = "https://www.allrecipes.com/air-fryer-miso-glazed-salmon-recipe-11842612"
result = analyze_recipe_from_url(recipe_url)

# Display results
display(Markdown(f"# {result['recipe']['title']}"))
display_nutrition(result['nutrition'])
if result['substitutions']:
    display_substitutions(result['substitutions'])


### Example 2: Analyze Recipe from Text

In [ ]:
# Example recipe text
sample_recipe = """
Classic Chocolate Chip Cookies

Ingredients:
- 2 1/4 cups all-purpose flour
- 1 teaspoon baking soda
- 1 cup butter, softened
- 3/4 cup granulated sugar
- 3/4 cup packed brown sugar
- 2 large eggs
- 2 teaspoons vanilla extract
- 2 cups chocolate chips

Instructions:
1. Preheat oven to 375°F
2. Mix flour and baking soda in a bowl
3. Cream butter and sugars until light and fluffy
4. Beat in eggs and vanilla
5. Gradually blend in flour mixture
6. Stir in chocolate chips
7. Drop rounded tablespoons onto ungreased cookie sheets
8. Bake 9-11 minutes until golden brown

Makes 48 cookies
"""

result = analyze_recipe_from_text(sample_recipe)
display(Markdown(f"# {result['recipe']['title']}"))
display_nutrition(result['nutrition'])
display_substitutions(result['substitutions'])


### Example 3: Modify Recipe for Dietary Restrictions


In [ ]:
# After analyzing a recipe, you can modify it for dietary restrictions

# Modify a recipe:
if result and result['recipe']:
    modified = modify_recipe_for_diet(result['recipe'], ["vegan", "gluten-free"])
    display_modified_recipe(modified)


### Complete Workflow Example

Here's a complete example showing the full pipeline from analysis to dietary modification:


In [ ]:
# COMPLETE WORKFLOW EXAMPLE
# Copy and modify this code to test with your own recipe

# Step 1: Analyze a recipe (from URL or text)
example_recipe = (
    "Pasta Carbonara\n\n"
    "Ingredients:\n"
    "- 400g spaghetti\n"
    "- 200g pancetta, diced\n"
    "- 4 large eggs\n"
    "- 100g parmesan cheese, grated\n"
    "- Black pepper\n"
    "- Salt\n\n"
    "Instructions:\n"
    "1. Cook pasta according to package directions\n"
    "2. Fry pancetta until crispy\n"
    "3. Beat eggs with parmesan and pepper\n"
    "4. Mix hot pasta with pancetta\n"
    "5. Add egg mixture, stirring quickly\n"
    "6. Serve immediately\n\n"
    "Serves 4"
)

# Analysis:
result = analyze_recipe_from_text(example_recipe)
if result:
    display(Markdown(f"# {result['recipe']['title']}"))
    display_nutrition(result['nutrition'])
    if result['substitutions']:
        display_substitutions(result['substitutions'])
    vegan_version = modify_recipe_for_diet(result['recipe'], ["vegan"])
    if vegan_version:
        display_modified_recipe(vegan_version)
